Section 1

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):

    df_da = (-3)*a**2 - (0.5)*(a**-0.5)
    df_db = (3)*cos(3*b) + (5/2)*(b**1.5)
    df_dc = 1/(c**2)

    return [df_da, df_db, df_dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO
# -----------
## a section
a = 2
b = 3
c = 4
h = 0.00000001


fa1 = f(a, b, c)
fa2 = f(a+h,b,c)

numerical_grad[0] = (fa2-fa1)/h

#b section
fb1 = f(a, b, c)
fb2 = f(a,b+h,c)

numerical_grad[1] = (fb2-fb1)/h

#c section
fc1 = f(a, b, c)
fc2 = f(a,b,c+h)

numerical_grad[2] = (fc2-fc1)/h


print(numerical_grad)
for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

[-12.353553380251014, 10.256990368162633, 0.0624999607623522]
OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


In [4]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [0, 0, 0] # TODO
# -----------

# -----------
## a section
a = 2
b = 3
c = 4
h = 0.00000001


fa1 = f(a-h, b, c)
fa2 = f(a+h,b,c)

numerical_grad2[0] = (fa2-fa1)/(2*h)

#b section
fb1 = f(a, b-h, c)
fb2 = f(a,b+h,c)

numerical_grad2[1] = (fb2-fb1)/(2*h)

#c section
fc1 = f(a, b, c-h)
fc2 = f(a,b,c + h)

numerical_grad2[2] = (fc2-fc1)/(2*h)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553291433172
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


## section 2: support for softmax

In [55]:
# Value class starter code, with many functions taken out
from math import log
import math

class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"
    

    def __radd__(self, other): #fallback for value + integer vs integer + value. 
        return self + other


    def __rmul__(self, other): #fallback for value * integer vs integer * value. If python values, it will try the other order using __rmul__ internal method
        return self * other
    
    def __truediv__(self, other): #Redefining division as an application of x**k, where k = -1. x**-1 = (1/x)
        return self * (other ** -1)
    
    def __neg__(self): #turns self negative
        return self * -1
    
    def __add__(self, other): # exactly as in the video
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        #set the backward function here
        out._backward = _backward

        return out
    
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other) #helps force data to be values in case we are working with Value, Integer operations
        out = Value(self.data*other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        #set the backward function here
        out._backward = _backward

        return out

    def tanh(self):
        n = self.data 
        t_val = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
        out = Value(t_val, (self, ), 'tanh')

        def _backward():
            self.grad += out.grad * (1 - t_val**2)

        #the backward function here
        out._backward = _backward
        return out
    
    def exp(self): 
        x = self.data 
        out  = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.grad * out.data #out.data is the derivative because d/dx(exp(x)) =  exp(x)
           

        #the backward function here
        out._backward = _backward
        return out
    
    def log(self):
        x = self.data
        out = Value(math.log(x), (self, ), 'log')

        def _backward():
            self.grad += ((x**-1))*out.grad

        out._backward = _backward
        return out


    def __pow__(self, other):
        assert isinstance(other, (int,float)), "only handle integer and float values for the calculation of power values"
        out = Value(self.data**other, (self, ), f'**{other}')

        def _backward():
            self.grad += (other*((self.data)**(other-1)))*out.grad #out.grad portion is reponsible for 'chaining' the calculation of the gradient

        out._backward = _backward
        return out


    def backward(self):
        """
        #The topological function here allows to chain together 
        #the calculation of backpward propagation/associated gradients across our connected nodes
        #This is creating a list of nodes such that each node is added after it's children are added.
        #Thus we calculated gradients of change backwards from children (right to left in topological format)
        """
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0 #initialize self.grad to 1.0
        for node in reversed(topo):
            node._backward() 



# ------
# re-implement all the other functions needed for the exercises below
# your code here
# TODO
# ------


##Manual example of gradient calculation
h = 0.0001
a = Value(2.0, label = 'a')
b = Value(-3.0, label = 'b')
c = Value(10.0, label = 'c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'

f = Value(-2.0, label = 'f')

L = d * f; L.label = 'L'
L1 = L.data


a = Value(2.0, label = 'a')
b = Value(-3.0 + h, label = 'b')
c = Value(10.0, label = 'c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label = 'f')

L = d * f; L.label = 'L'
L2 = L.data

print((L2 - L1)/h)

-4.000000000008441


### Neuron Example

In [6]:
#input values
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

#weights
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

#bias of the neuron
b = Value(6.88137, label = "b")

x1w1 = x1*w1; label = 'x1*w1'
x2w2 = x2*w2; label = 'x2*w2'

#full input
full_input = x1w1 + x2w2; full_input.label = "x1*w1 + x2*w2"
n = full_input + b; n.label = 'neuron'

o = n.tanh(); o.label = 'output'

In [7]:
#make sure we initialize o to 1.0 before running.backward()
o.backward()


In [8]:
#what should n grad be?
n.grad #default should be 0.5. Why?

#what should o grad be?
o.grad 


1.0

## Gradient experiment


In [9]:
o.grad = 1.0
o._backward()

In [10]:
n._backward()
n.grad

1.000005072818119

In [11]:
b._backward()
b.grad

1.5000076092271784

In [12]:
full_input._backward()
full_input.grad

1.5000076092271784

In [13]:
x2w2._backward()
x2w2.grad

2.000010145636238

In [14]:
x1w1._backward()
x1w1.grad

2.000010145636238

In [15]:
x1._backward()
x1.grad

-7.5000380461358915

In [16]:
w1._backward()
w1.grad

5.000025364090595

In [17]:
x2._backward()
x2.grad

2.5000126820452975

In [18]:
w2._backward()
w2.grad

0.0

In [19]:
#input values
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

#weights
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

#bias of the neuron
b = Value(6.88137, label = "b")

x1w1 = x1*w1; label = 'x1*w1'
x2w2 = x2*w2; label = 'x2*w2'

#full input
full_input = x1w1 + x2w2; full_input.label = "x1*w1 + x2*w2"
n = full_input + b; n.label = 'neuron'

o = n.tanh(); o.label = 'output'

In [56]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
print(probs)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


### Torch exploration

In [68]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import random

In [66]:
#input values
x1 = torch.Tensor([2.0]).double(); x1.requires_grad = True
x2 = torch.Tensor([0.0]).double(); x2.requires_grad = True

#weights
w1 = torch.Tensor([-3.0]).double(); w1.requires_grad = True
w2 = torch.Tensor([1.0]).double(); w2.requires_grad = True


#bias of the neuron
b = torch.Tensor([6.8813735870195432]).double(); b.requires_grad = True
full_input = x1*w1 + x2*w2
n = full_input + b; n.label = 'neuron'
o = torch.tanh(n)


#### Neuron, Layer and MLP in micrograd

In [143]:
class Neuron:

    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for i in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x): #__call__ provides functionality applied at n(x)

        #Create a zipped iterable for w, x for dot product wise calculation
        neural_activation = sum(w_pos * x_pos for w_pos, x_pos in zip(self.w, x)) + self.b
        neural_response = neural_activation.tanh() #nonlinearity function for response

        return neural_response
    
    def parameters(self):
        return self.w + [self.b]
    

class Layer:
    def __init__(self, nin, neurons_in_layer): 
        self.neurons = [Neuron(nin) for i in range(neurons_in_layer)]
        #neurons in layer is the number of independent neurons we want in this layer.
        #nin denotes the number of inputs each neuron in this layer should take.

    def __call__(self, x):
        outs = [n(x) for n in self.neurons] #the list of neural responses
        
        return outs[0] if len(outs) == 1 else outs #return single element value if outs is a single element. Else out.
    
    def parameters(self):
        #return [p for neuron in self.neurons for p in neuron.parameters()]
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params


class MLP:
    def __init__(self, nin, nouts): #nouts must a list that signifies the number of neurons we want in each layer
        size = [nin] + nouts 
        self.layers = [Layer(size[i], size[i+1]) for i in range(len(nouts))] 

    def __call__(self, x): # we want this function to create our layers within the MLP when we call it
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        params = []
        for layer in self.layers:
            ps = layer.parameters()
            params.extend(ps)

        return ps
    
# x = [2.3, 3.0] #inputs into the neuron
# n = Neuron(2)
# n(x)

# x = [-4, 5]
# n = Layer(2, 3)

# n(x)

x = [3, 5, 4, 3, 7,10]
n = MLP(6, [4,4,1])
n(x)

Value(data=-0.046540483011478236)

### Sample execution of the data

In [117]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0]

ypred = [n(x) for x in xs]
ypred

[Value(data=-0.36007679879186466),
 Value(data=0.4857981152710343),
 Value(data=0.30140283050505345),
 Value(data=0.333976222691408)]

In [118]:
loss = sum((yout - ygt)**2 for yout, ygt in zip(ypred, ys))
loss

Value(data=6.194641937141853)

We're now going to do backward propagation and calculate the gradients of each neuron in each layer. These gradients are then used to adjust the weights for each neuron

In [119]:
loss.backward()

In [120]:
#How many weights are on the 2nd neuron of the first layer
len(n.layers[0].neurons[1].w)

6

In [121]:
#What is the gradient of the 3rd weight of the 2nd neuron of the first layer?
#negative grads would mean that increasing this weight will make the loss go down. Influence on loss is negative (numerical)
#positive grads would mean that increasing this weight will make the loss go up. Influence on loss is positive (numerical)
n.layers[0].neurons[1].w[2].grad

-2.1469797157632837

### Now we want to adjust weights using gradient descent

In [134]:
for p in n.parameters():
    p.data += -0.01 * p.grad #minimize the loss. REal number is the learning rate

In [135]:
ypred = [n(x) for x in xs]
ypred

[Value(data=-0.24063122334875564),
 Value(data=-0.11629472115993122),
 Value(data=0.06803411789194198),
 Value(data=0.09521259133832294)]

In [136]:
loss = sum((yout - ygt)**2 for yout, ygt in zip(ypred, ys))
loss

Value(data=4.279437984051564)

In [137]:
c

#### Full training loop by hand with micrograd

In [144]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0]

In [145]:
for k in range(20):
    if k % 5 == 0:
        print(k, ' position through loop')

    #Forward pass
    ypred = [n(x) for x in xs]
    #loss calculation within the forward pass
    loss = sum((yout - ygt)**2 for yout, ygt in zip(ypred, ys))
    
    # #loss break
    # if loss.data < .5:
    #     break

    """#MAKE SURE TO FLUSH the grad.
        If we don't flush the grads then the model doesn't learn anything.
        It's just adding grads from different loops
    """

    for p in n.parameters():
       p.grad = 0.0

    #backward pass to calculate grads from forward pass. Grads tell us how changing the weight mattered!
    loss.backward()


    for p in n.parameters():#MAKE SURE TO FLUSH the grad
        p.data += -0.05 * p.grad

    print(k, loss.data)


0  position through loop
0 4.339661626111346
1 4.200537490613689
2 4.072553120756764
3 3.928985589583614
4 3.764181918058666
5  position through loop
5 3.5733751340984545
6 3.353000859143111
7 3.1029291264379375
8 2.829193749096187
9 2.545069739021252
10  position through loop
10 2.268008312506775
11 2.013428880326604
12 1.790185060781662
13 1.6003574569760963
14 1.4416042497128316
15  position through loop
15 1.309621221351183
16 1.1997420647173411
17 1.1077182709847535
18 1.0299860204173767
19 0.9636785424312667


# With Torch for softmax

In [148]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch.nn.functional as F

In [157]:

logits = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double(); logits.requires_grad = True
gradient = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]

# softmax_probs = F.softmax(logits)
# print(probs)
# loss = -softmax_probs[3].log()
# print(loss, '---- loss')

loss = -F.log_softmax(logits, dim = -1)[3]
print(loss)

#now update the gradients
loss.backward()

print("Latest Loss: ", loss.data.item())

print("Grads: ", logits.grad)


tensor(2.1755, dtype=torch.float64, grad_fn=<NegBackward0>)
Latest Loss:  2.1755153626167143
Grads:  tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)
